In [1]:
from batid.services.guess_bdg_new import Guesser,GeocodeAddressHandler
from batid.models import Building
import csv
import os

os.environ["DJANGO_ALLOW_ASYNC_UNSAFE"] = "true"

In [2]:
def row_to_input(line_number, row) -> dict:
    
    address_infos = [
        row['num_voie'],
        row['ind_rep'],
        row['nom_voie'],
        row['code_postal'],
        row['commune']
    ]
    
    address_infos = [i for i in address_infos if i]
    address = " ".join(address_infos)
    
    return {
        'ext_id': f"{row['num_arrete']}--ligne{line_number}",
        'address': address
    }
    

In [3]:
os.remove("guess.json")

with open("historique_insa_rnb.csv", "r") as f:
    
    reader = csv.DictReader(f)
    inputs = []
    
    for i, row in enumerate(list(reader)):
        inputs.append(row_to_input(i+2, row))
        
    
    guesser = Guesser()
    guesser.load_inputs(inputs)
    guesser.save_work_file('guess.json')

In [4]:
guesser = Guesser()


guesser.handlers = [GeocodeAddressHandler()]
guesser.guess_work_file('guess.json')

guesser.report()




-- Report --
Number of rows: 277
Number of match: 225 (81.23%)

-- match_reasons : absolute --
precise_address_match    225
Name: match_reason, dtype: int64

-- match_reasons : % --
precise_address_match    81.227437
Name: match_reason, dtype: float64

-- Inputs --
rows with ban_id: 228 (82.31%)


In [5]:
guesser.display_unmatched(30)

    matches match_reason     finished_steps        input_ext_id  \
92                  None  [geocode_address]    2022-54--ligne94   
56                  None  [geocode_address]    2022-06--ligne58   
128                 None  [geocode_address]   2022-93--ligne130   
43                  None  [geocode_address]    2021-67--ligne45   
182                 None  [geocode_address]   2023-25--ligne184   
96                  None  [geocode_address]    2022-58--ligne98   
77                  None  [geocode_address]    2022-29--ligne79   
129                 None  [geocode_address]   2022-94--ligne131   
153                 None  [geocode_address]  2022-132--ligne155   
257                 None  [geocode_address]  2023-116--ligne259   
199                 None  [geocode_address]   2023-42--ligne201   
67                  None  [geocode_address]    2022-18--ligne69   
246                 None  [geocode_address]   2023-99--ligne248   
17                  None  [geocode_address]    2021-21--ligne1

In [6]:
bdgs = Building.objects.filter(addresses__id='13054_0530_00007')
print(bdgs.count())

1


In [7]:
guesser = Guesser()
guesser.load_work_file('guess.json')
guesser.to_csv('marseille-insalubrite-rnb.csv', ext_id_col_name="Identifiant arrêté")